In [16]:
import numpy as np
from itertools import product
from dataclasses import dataclass
import matplotlib.pyplot as plt

Ref: https://github.com/jomavera/optimizationAlgosPort/blob/main/chap_14.ipynb

# Algorithm 14.5 + 14.6

In [17]:
def radial_bases(psi, C, p=2):
    '''
    - psi: a radial function
    - C: list of centers
    - p: L_p norm (default: L2)
    '''
    bases = [] # list of basis function, containing b_i(x)
    for c in C:
        func = lambda x, c=c: psi(np.linalg.norm(x-c, p)) # psi(||x-x^(i)||), với c = x^(i)
        # def func(x,c=c):
        #     return psi(np.linalg.norm(x-c,p))
        
        bases.append(func)
    return bases

def regression(X, y, bases, lamb):
    B = np.array([[b(x) for b in bases] for x in X]) # basis matrix
    theta = np.linalg.inv((B.T @ B + lamb * np.identity(len(bases)))) @ B.T @ y
    # theta = np.linalg.solve(np.matmul(B.T,B) + np.diag(lamb*np.ones((len(bases),1))),np.matmul(B.T,y))
    return lambda x: np.sum([theta[i]*bases[i](x) for i in range(len(theta))])

## Algorithm 14.7:

In [18]:
@dataclass
class TrainTest:
    train: list
    test: list
    
def train_and_validate(X, y, tt, fit, metric):
    model = fit(X[tt.train], y[tt.train])
    return metric(model, X[tt.test], y[tt.test])

# TEST

In [19]:
def polynomial_bases_1d(i,k): # polynomial cho 1 cột của x: 1,x,x^2,x^3,...
    f = lambda x: np.power(x[i], np.arange(0,k+1,1))
    return f

def polynomial_bases(n,k):
    bases = [polynomial_bases_1d(i, k) for i in range(n)]
    terms = []
    i = 0
    
    for ks in product(*[np.arange(0,k+1) for _ in range(n)]): # cartesian product: [[0,0], [0,1],...,(3,3)]
        if np.sum(ks) <= k: # DK trong CT 14.19: i+j<=k
            def func(x, ks=ks):
                res = np.prod([b(x)[j] for j, b in zip(ks,bases)]) # CT 14.19 tính b_{ij}(x) = x_1^i * x_2^j
                # print(x, ks, res)
                return res 
            terms.append(func)
            
            # print('terms: ', i)
            # i+=1

    return terms

In [20]:
##
n, k = 2, 3 # 1 data point có 2 cột
x = [2,3]
bases = [polynomial_bases_1d(i, k) for i in range(n)] # với từng thuộc tính (col), mũ hóa nó lên

for ks in product(*[np.arange(0,k+1) for i in range(n)]): # loop qua từng mũ có thể có: [[0,0], [0,1],...,(3,3)]
    if np.sum(ks) <= k:
        print(f'ks: {ks}')

        l = np.prod([b(x)[j] for j, b in zip(ks,bases)]) # tính b_{ij}(x) = x_1^i * x_2^j, lưu trong terms, chú ý term khác với parameter theta
        print(l)
        # break
    # break
    # print(np.sum(ks))
    

ks: (0, 0)
1
ks: (0, 1)
3
ks: (0, 2)
9
ks: (0, 3)
27
ks: (1, 0)
2
ks: (1, 1)
6
ks: (1, 2)
18
ks: (2, 0)
4
ks: (2, 1)
12
ks: (3, 0)
8


In [27]:
np.random.seed(35912)
bases = polynomial_bases(2,3)
X = np.array([[1.0, 5.0],
              [2.0, 2.0],
              [3.0, 2.0],
              [4.0, 2.0]])

y = np.array([2, 2, 5, 8]).reshape(-1,1)

lmb=[0.3 for i in range(len(bases))]
model = regression(X,y,bases,lmb)
print(model([4,6,7]))

35.52256700620834


In [ ]:
len(bases) # do có 10 ks thỏa <= k

10